# 车牌识别大作业

李沿橙 1900012766

In [1]:
# import collection
import cv2
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import matplotlib.patches as patches

import visdom
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
torch.manual_seed(2)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## 一、数据准备

In [2]:
# 获取一个目录下所有文件的文件名（包含 path），返回一个列表，按字典序返回
def get_all_files(path):
    return ["\\".join([path, x]) for x in os.listdir(path)]

In [3]:
root_path_train = r'.\Plate_dataset\AC\train\xml' # 训练集目录
root_path_test = r'.\Plate_dataset\AC\test\xml'

In [4]:
# 根据 path 指定的 xml 文件，返回一个二元组，第一个元素是裁剪后的车牌图片，第二个是车牌号码
def xml_to_pair(path):
    xml_loader = ET.ElementTree(file = path)
    second = xml_loader.find('object').find('platetext').text
    
    filename = xml_loader.find('filename').text
    filename = '\\'.join(path.split('\\')[:-2]) + '\\jpeg\\' + filename # 图片的完整路径

    img = cv2.imread(filename)
    bbox = xml_loader.find('object').find('bndbox')
    xmin = bbox.find('xmin').text
    ymin = bbox.find('ymin').text
    xmax  = bbox.find('xmax').text
    ymax = bbox.find('ymax').text
    bbox = [xmin, ymin, xmax, ymax]
    bbox = [int(b) for b in bbox] # left, top, right, bottom
    img = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]

    first = img
    #first = cv2.resize(img, (80, 40))
    return (first, second)

In [5]:
root_path_char = r'.\Chars_data'

In [6]:
# 返回路径的最后一个字符
def path_to_char(path):
    return path[-1]

In [7]:
# 根据 path 指定的 char 的文件夹，返回一个二元组的列表，第一个元素是图片，第二个元素是字符的 ASCII 码
def path_ch_to_pairs(path):
    ch = ord(path_to_char(path))
    return [(cv2.imread(x, flags = cv2.IMREAD_GRAYSCALE), ch) for x in get_all_files(path)]

In [8]:
# 得到处理后的原始数据
raw_train = [xml_to_pair(x) for x in get_all_files(root_path_train)] # [(img, str)]
raw_test = [xml_to_pair(x) for x in get_all_files(root_path_test)] # [(img, str)]
raw_chars = { path_to_char(path_ch): path_ch_to_pairs(path_ch) for path_ch in [x for x in get_all_files(root_path_char)] } # {[(img, ascii)]}

## 二、数据预处理